
<img src="https://storage.googleapis.com/unskript-website/assets/favicon.png" alt="unSkript.com" width="100" height="100"/> 
<h1> unSkript Runbooks </h1>
<div class="alert alert-block alert-success">
    <b> This runbook demonstrates How to Resize PVC attached to a K8S Cluster.
    We use unSkript k8s Actions to achieve this complex tax.</b>
</div>

<br>
<center><h2>Resize PVC</h2></center>

# Steps Overview 
1 <a href="#1"> Get Storageclass associated with the PVC </a>
<br>
2 <a href="#2"> Verify that allowVolumeExpansion is enabled </a>
<br>
3 <a href="#3"> Increase the PVC Volume by the provided Value depending upon ResizeOption chosen. </a>
<br>
4 <a href="#4"> Find out POD Name related to the PVC </a>
<br>
4.1 <a href="#41"> Restart POD if RestartPodAfterResize is set to True </a>
<br>
    (NOTE: This is not required if the kubernetes has ExpandInUsePersistentVolumes enabled.)
<br>
5 <a href="#5"> Verify resize by running 'df -kh' on the pod attached to the PVC. </a>
<br>
5.1 <a href="#51"> Final Verification </a>

<p id="1"> </p>

### Get Storageclass associated with the PVC

Here we use the `kubectl get sc` to get the storageclass for the given PVC. This will be stored in
the varaible `storageClass`. 

In [4]:
#
# Copyright (c) 2021 unSkript.com
# All rights reserved.
#

from pydantic import BaseModel, Field


from beartype import beartype
@beartype
def k8s_kubectl_command(handle, kubectl_command: str) :

    assert(kubectl_command.startswith("kubectl"))

    result = handle.run_native_cmd(kubectl_command)
    if result.stderr:
        print(
            f"Error while executing command ({kubectl_command}): {result.stderr}")
        return None

    return result.stdout




task = Task(Workflow())
task.configure(inputParamsJson='''{
    "kubectl_command": "f\\"kubectl get pvc {PVCName} -n {Namespace} --output=jsonpath={{.spec.storageClassName}}\\""
    }''')
task.configure(outputName="storageClass")

(err, hdl, args) = task.validate(vars=vars())
if err is None:
    task.output = task.execute(k8s_kubectl_command, hdl, args)
if hasattr(task, 'output'):
    print(task.output)

<p id="2"> </p>

### Verify alloVolumeExpansion is enabled

Here we check if `allowVolumeExpansion` is enabled for the storage class for the given PVC. If the PVC does not
have this enabled, the execution would assert.

In [5]:
#
# Copyright (c) 2021 unSkript.com
# All rights reserved.
#

from pydantic import BaseModel, Field


from beartype import beartype
@beartype
def k8s_kubectl_command(handle, kubectl_command: str) :

    assert(kubectl_command.startswith("kubectl"))

    result = handle.run_native_cmd(kubectl_command)
    if result.stderr:
        print(
            f"Error while executing command ({kubectl_command}): {result.stderr}")
        return None

    return result.stdout




task = Task(Workflow())
task.configure(inputParamsJson='''{
    "kubectl_command": "f\\"kubectl get sc {storageClass} --output=jsonpath={{.allowVolumeExpansion}}\\""
    }''')
task.configure(outputName="allowVolumeExpansion")

(err, hdl, args) = task.validate(vars=vars())
if err is None:
    task.output = task.execute(k8s_kubectl_command, hdl, args)
if hasattr(task, 'output'):
    print(task.output)

In [6]:
if allowVolumeExpansion == "" or allowVolumeExpansion is False:
    print(f'allowVolumeExpansion disabled for storage class {storageClass}, exiting')
    assert(f'allowVolumeExpansion disabled for storage class {storageClass}')
else:
    print(f'allowVolumeExpansion enabled for storage class {storageClass}')

<p id="3"> </p>

### Increase the PVC Volume 

Here we will use unSkript `k8s_change_pvc_size` Action to increase the PVC volume to the  provided Value depending upon ResizeOption chosen.

In [2]:
#
# Copyright (c) 2021 unSkript.com
# All rights reserved.
#

import pprint
from pydantic import BaseModel, Field
from typing import Optional
from unskript.enums.aws_k8s_enums import SizingOption

from beartype import beartype

def k8s_change_pvc_size_printer(output):
    if output is None:
        return

    pprint.pprint(output)


@beartype
def k8s_change_pvc_size(handle, namespace: str, name: str, resize_option: SizingOption, resize_value: float) -> str:
    # Get the current size.
    kubectl_command = f'kubectl get pvc {name} -n {namespace}  -o jsonpath={{.status.capacity.storage}}'
    result = handle.run_native_cmd(kubectl_command)
    if result.stderr:
        print(
            f"Error while executing command ({kubectl_command}): {result.stderr}")
        return None

    currentSize = result.stdout
    currentSizeInt = int(currentSize.rstrip("Gi"))
    if resize_option == SizingOption.Add:
        newSizeInt = currentSizeInt + resize_value
    else:
        newSizeInt = currentSizeInt * resize_value
    newSize = str(newSizeInt) + "Gi"
    print(f'Current size {currentSize}, new Size {newSize}')
    kubectl_command = f'kubectl patch pvc {name} -n {namespace} -p \'{{"spec":{{"resources":{{"requests": {{"storage": "{newSize}"}}}}}}}}\''
    result = handle.run_native_cmd(kubectl_command)
    if result.stderr:
        print(
            f"Error while executing command ({kubectl_command}): {result.stderr}")
        return None
    print(f'PVC {name} size changed to {newSize} successfully')
    return result.stdout

task = Task(Workflow())
task.configure(inputParamsJson='''{
    "name": "PVCName",
    "namespace": "Namespace",
    "resize_option": "SizingOption.Add if ResizeOption == \\"Add\\" else SizingOption.Mutiple",
    "resize_value": "float(Value)"
    }''')
(err, hdl, args) = task.validate(vars=vars())
if err is None:
    task.execute(k8s_change_pvc_size, lego_printer=k8s_change_pvc_size_printer, hdl=hdl, args=args)

<p id="4"> </p>

### Find out the POD related to the PVC

Here we discover the POD name related to the PVC. This will be important for our Next step where
we may have to `restart` the POD depending on the `RestartPodsAfterResize` being enabled or not. This
cell will store the Pod Name in the variable `podName`

In [ ]:
#
# Copyright (c) 2021 unSkript.com
# All rights reserved.
#

from pydantic import BaseModel, Field
from typing import Optional
import enum

from beartype import beartype
@beartype
def k8s_get_pods_attached_to_pvc(handle, namespace: str, pvc: str) :
    kubectl_command = f"kubectl describe pvc {pvc} -n {namespace} | awk \'/Used By/ {{print $3}}\'"
    result = handle.run_native_cmd(kubectl_command)
    if result.stderr:
        print(
            f"Error while executing command ({kubectl_command}): {result.stderr}")
        return None
    return result.stdout




task = Task(Workflow())
task.configure(inputParamsJson='''{
    "namespace": "Namespace",
    "pvc": "PVCName"
    }''')
task.configure(outputName="podName")
(err, hdl, args) = task.validate(vars=vars())
if err is None:
    task.output = task.execute(k8s_get_pods_attached_to_pvc, hdl, args)
if hasattr(task, 'output'):
    print(task.output)

In [ ]:
podName = podName.strip()
print(f'Pod {podName} attached to PVC {PVCName}')

<p id="41"> </p>

### Restart POD if RestartPodsAfterResize enabled

Here we use the `kubectl delete pod` command to delete the POD that was identified in the previous step.
Once deleted, K8S will restart the pod and hence achieve the objective. This cell makes use of unSkript `Start Condition`
Feature to execute only when it finds out `RestartPodsAfterResize` is set to `True`. 

In [ ]:
#
# Copyright (c) 2021 unSkript.com
# All rights reserved.
#

from pydantic import BaseModel, Field


from beartype import beartype
@beartype
def k8s_kubectl_command(handle, kubectl_command: str) :

    assert(kubectl_command.startswith("kubectl"))

    result = handle.run_native_cmd(kubectl_command)
    if result.stderr:
        print(
            f"Error while executing command ({kubectl_command}): {result.stderr}")
        return None

    return result.stdout




task = Task(Workflow())
task.configure(inputParamsJson='''{
    "kubectl_command": "f\\"kubectl delete pod {podName} -n {Namespace}\\""
    }''')
task.configure(conditionsJson='''{
    "condition_enabled": true,
    "condition_cfg": "RestartPodsAfterResize==True",
    "condition_result": true
    }''')

(err, hdl, args) = task.validate(vars=vars())
if err is None:
    task.output = task.execute(k8s_kubectl_command, hdl, args)
if hasattr(task, 'output'):
    print(task.output)

<p id="5"> </p>

### Verify resize by running 'df -kh' on the pod attached to the PVC.

In this cell, we will use the `df -k` command on the POD to verify if resize was done successfuly or not.

In [ ]:
#
# Copyright (c) 2021 unSkript.com
# All rights reserved.
#

from typing import List

from kubernetes import client
from kubernetes.stream import stream
from pydantic import BaseModel, Field


from beartype import beartype
@beartype
def k8s_exec_command_on_pod(handle, namespace: str, podname: str, command: List) :
    coreApiClient = client.CoreV1Api(api_client=handle)

    try:
        resp = stream(coreApiClient.connect_get_namespaced_pod_exec,
                      podname,
                      namespace,
                      command=list(command),
                      stderr=True,
                      stdin=True,
                      stdout=True,
                      tty=False
                      )
    except:
        resp = 'An Exception occured while executing the command'

    return resp




task = Task(Workflow())
task.configure(inputParamsJson='''{
    "command": "[\\"df\\", \\"-kh\\"]",
    "namespace": "Namespace",
    "podname": "podName"
    }''')

(err, hdl, args) = task.validate(vars=vars())
print(args)
if err is None:
    task.output = task.execute(k8s_exec_command_on_pod, hdl, args)
if hasattr(task, 'output'):
    print(task.output)

<p id="51"> </p>

### Final verification 

Here, we will use the `kubectl get pvc` to query the modified storage size. The result is printed 
for user to verify if the value is set properly.

In [ ]:
#
# Copyright (c) 2021 unSkript.com
# All rights reserved.
#

from pydantic import BaseModel, Field


from beartype import beartype
@beartype
def k8s_kubectl_command(handle, kubectl_command: str) :

    result = handle.run_native_cmd(kubectl_command)
    if result is None or hasattr(result, "stderr") is False or result.stderr is None:
        print(
            f"Error while executing command ({kubectl_command}): {result.stderr}")
        return None

    return result.stdout


task = Task(Workflow())
task.configure(inputParamsJson='''{
    "kubectl_command": "f\\"kubectl get pvc {PVCName} -n {Namespace}  -o jsonpath={{.status.capacity.storage}}\\""
    }''')
task.configure(outputName="newSize")

(err, hdl, args) = task.validate(vars=vars())
if err is None:
    task.output = task.execute(k8s_kubectl_command, hdl, args)
if hasattr(task, 'output'):
    if isinstance(task.output, (list, tuple)):
        for item in task.output:
            print(f'item: {item}')
    elif isinstance(task.output, dict):
        for item in task.output.items():
            print(f'item: {item}')
    else:
        print(task.output)

### Conclusion

In this runbook, we used unSkript `Start Condition`, `Chaining` and `Custom Cell` features to construct a complex Runbook such as Resizing PVC for a given K8S Cluster.  

<br>

To learn more about the `unSkript` platform, Please visit https://unskript.com 